In [1]:
import pandas as pd
import numpy as np
import re
import csv
import pprint
from collections import Counter
import dedupe

In [2]:
# Read original csv file
df = pd.read_csv('restaurant-training.csv', skip_blank_lines=True)
df = df.drop(columns=['phone', 'cluster'])
df = df.fillna('')
# Add an id column to each row
df['id'] = list(range(len(df)))
df

,name,addr,city,type,id
0,locanda veneta,3rd st.,los angeles,italian,0
1,locanda veneta,8638 w. third st.,los angeles,italian,1
2,locanda veneta,8638 w 3rd,st los angeles,italian,2
3,cafe lalo,201 w. 83rd st.,new york,coffee bar,3
4,cafe lalo,201 w. 83rd st.,new york city,coffeehouses,4
5,les celebrites,160 central park s,new york,french,5
6,les celebrites,155 w. 58th st.,new york city,french (classic),6
7,second avenue deli,156 2nd ave. at 10th st.,new york,delicatessen,7
8,second avenue deli,156 second ave.,new york city,delis,8
9,smith & wollensky,201 e. 49th st.,new york,american,9


In [3]:
irrelevant_regex = re.compile(r'[^a-z0-9\s]')
multispace_regex = re.compile(r'\s\s+')

In [4]:
# Cleaning the name column
def assign_no_symbols_name(df):
    return df.assign(
        name=df['name']
             .str.replace(irrelevant_regex, ' ', regex=True)
             .str.replace(multispace_regex, ' ', regex=True))

In [5]:
possible_stopwords = Counter(' '.join(df['name']).split()).most_common(20)
pprint.pprint(possible_stopwords)

[('cafe', 5),
 ('locanda', 3),
 ('veneta', 3),
 ('bistro', 3),
 ('lalo', 2),
 ('les', 2),
 ('celebrites', 2),
 ('second', 2),
 ('avenue', 2),
 ('deli', 2),
 ('smith', 2),
 ('&', 2),
 ('wollensky', 2),
 ("chin's", 2),
 ('toulouse', 2),
 ('rose', 2),
 ('pistola', 2),
 ('remi', 2),
 ('stars', 2),
 ('garden', 1)]


In [6]:
# Deleting stopwords
def assign_cleaned_name(df):
    restaurant_stopwords = {
        's', 'the', 'la', 'le', 'of', 'and', 'on', 'l'}
    restaurant_stopwords_regex = r'\b(?:{})\b'.format(
        '|'.join(restaurant_stopwords))
    return df.assign(
        name=df['name']
             .str.replace(restaurant_stopwords_regex, '', regex=True)
             .str.replace(multispace_regex, ' ', regex=True)
             .str.strip())

In [7]:
df = assign_no_symbols_name(df)
df = assign_cleaned_name(df)
df.head(5)

,name,addr,city,type,id
0,locanda veneta,3rd st.,los angeles,italian,0
1,locanda veneta,8638 w. third st.,los angeles,italian,1
2,locanda veneta,8638 w 3rd,st los angeles,italian,2
3,cafe lalo,201 w. 83rd st.,new york,coffee bar,3
4,cafe lalo,201 w. 83rd st.,new york city,coffeehouses,4


In [8]:
# Dedupe configuration
fields = [
    {
        'field': 'name',
        'variable name': 'name',
        'type': 'ShortString',
        'has missing': True
    },
    {
        'field': 'addr',
        'variable name': 'addr',
        'type': 'ShortString',
    },
    {
        'field': 'city',
        'variable name': 'city',
        'type': 'ShortString',
        'has missing': True
    },
]

In [9]:
# Deduper initialization
deduper = dedupe.Dedupe(fields, num_cores=4)

In [10]:
# Formating the data for dedupe
data_for_dedupe = df.to_dict('index')
for record in data_for_dedupe.values():
    # Change nans to None
    for k, v in record.items():
        if isinstance(v, float) and np.isnan(v):
            record[k] = None

In [11]:
deduper.prepare_training(data_for_dedupe)

In [12]:
# Manual labeling
dedupe.console_label(deduper)

name : rose pistola
addr : 532 columbus ave.
city : san francisco

name : rose pistola
addr : 532 columbus ave.
city : san francisco

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished
name : chin
addr : 3200 las vegas blvd. s
city : las vegas

name : chin
addr : 3200 las vegas blvd. s.
city : las vegas

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious
name : toulouse
addr : b peachtree rd.
city : atlanta

name : toulouse
addr : 293-b peachtree rd.
city : atlanta

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious
name : second avenue deli
addr : 156 2nd ave. at 10th st.
city : new york

name : second avenue deli
addr : 156 second ave.
city : new york city

3/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious
name : chin
ad

In [13]:
# Begin with the trainig
deduper.train()

In [14]:
# Shows the rules found during the training
deduper.predicates

(SimplePredicate: (suffixArray, name),
 SimplePredicate: (wholeFieldPredicate, addr))

In [15]:
# Clustering

# `partition` will return sets of records that dedupe
# believes are all referring to the same entity.

print('clustering...')
clustered_dupes = deduper.partition(data_for_dedupe, 0.5)

print('# duplicate sets', len(clustered_dupes))

clustering...
# duplicate sets 27


In [16]:
clustered_dupes

[((1, 2), array([0.78602743, 0.78602743])),
 ((3, 4), (0.98323005, 0.98323005)),
 ((7, 8), (0.8077531, 0.8077531)),
 ((11, 12), (0.9838027, 0.9838027)),
 ((13, 14), (0.93425727, 0.93425727)),
 ((15, 16), (0.9846, 0.9846)),
 ((0,), (1.0,)),
 ((5,), (1.0,)),
 ((6,), (1.0,)),
 ((9,), (1.0,)),
 ((10,), (1.0,)),
 ((17,), (1.0,)),
 ((18,), (1.0,)),
 ((19,), (1.0,)),
 ((20,), (1.0,)),
 ((21,), (1.0,)),
 ((22,), (1.0,)),
 ((23,), (1.0,)),
 ((24,), (1.0,)),
 ((25,), (1.0,)),
 ((26,), (1.0,)),
 ((27,), (1.0,)),
 ((28,), (1.0,)),
 ((29,), (1.0,)),
 ((30,), (1.0,)),
 ((31,), (1.0,)),
 ((32,), (1.0,))]

In [17]:
# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other with a confidence score.

cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            'Cluster ID': cluster_id,
            'confidence_score': score
        }

In [20]:
cluster_membership

{1: {'Cluster ID': 0, 'confidence_score': 0.7860274291465319},
 2: {'Cluster ID': 0, 'confidence_score': 0.7860274291465319},
 3: {'Cluster ID': 1, 'confidence_score': 0.98323005},
 4: {'Cluster ID': 1, 'confidence_score': 0.98323005},
 7: {'Cluster ID': 2, 'confidence_score': 0.8077531},
 8: {'Cluster ID': 2, 'confidence_score': 0.8077531},
 11: {'Cluster ID': 3, 'confidence_score': 0.9838027},
 12: {'Cluster ID': 3, 'confidence_score': 0.9838027},
 13: {'Cluster ID': 4, 'confidence_score': 0.93425727},
 14: {'Cluster ID': 4, 'confidence_score': 0.93425727},
 15: {'Cluster ID': 5, 'confidence_score': 0.9846},
 16: {'Cluster ID': 5, 'confidence_score': 0.9846},
 0: {'Cluster ID': 6, 'confidence_score': 1.0},
 5: {'Cluster ID': 7, 'confidence_score': 1.0},
 6: {'Cluster ID': 8, 'confidence_score': 1.0},
 9: {'Cluster ID': 9, 'confidence_score': 1.0},
 10: {'Cluster ID': 10, 'confidence_score': 1.0},
 17: {'Cluster ID': 11, 'confidence_score': 1.0},
 18: {'Cluster ID': 12, 'confidence_sc

In [23]:
# To use the id column
df.to_csv('restaurant-training-opt.csv', index=False)

In [24]:
input_file = 'restaurant-training-opt.csv'
output_file = 'restaurant_training_output.csv'

In [26]:
# Writing the results
with open(output_file, 'w') as f_output, open(input_file) as f_input:
    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)